# Autosome model run example

We added submodule `autosome` to the `prixfixe` module with the implementation of all required modules.



The data processing step can be very specialized for particular approach(es), so `DataProcessors` from different teams require lots of arguments. Maybe it can be simplified once we know which teams adapt their code for the post-challenge analysis.

In [ ]:
TRAIN_DATA_PATH = "/home/dnogina/dreamdata/contest_data/train.txt"
VALID_DATA_PATH = "/home/dnogina/dreamdata/contest_data/val.txt"
TRAIN_BATCH_SIZE = 1024
BATCH_PER_EPOCH = 1000
N_PROCS = 8
VALID_BATCH_SIZE = 4096
BATCH_PER_VALIDATION = 125 
PLASMID_PATH = "plasmid.json"
SEQ_SIZE = 150

In [ ]:
import torch
from prixfixe.autosome import AutosomeDataProcessor

generator = torch.Generator()
generator.manual_seed(2147483647)

dataprocessor = AutosomeDataProcessor(
    path_to_training_data=TRAIN_DATA_PATH,
    path_to_validation_data=VALID_DATA_PATH,
    train_batch_size=TRAIN_BATCH_SIZE, 
    batch_per_epoch=BATCH_PER_EPOCH,
    train_workers=N_PROCS,
    valid_batch_size=VALID_BATCH_SIZE,
    batch_per_valid=BATCH_PER_VALIDATION,
    valid_workers=N_PROCS,
    shuffle_train=True,
    shuffle_val=False,
    plasmid_path=PLASMID_PATH,
    seqsize=SEQ_SIZE,
    generator=generator
)

The creation of the model is very simple. 

The only thing you should notice is the requirement for `DataProcessor` to have a `data_channels` and `data_seqsize` methods. Otherwise, it is impossible to configure `FirstLayerBlock` correctly.


In [ ]:
from prixfixe.autosome import (AutosomeCoreBlock,
                      AutosomeFirstLayersBlock,
                      AutosomeFinalLayersBlock)

from prixfixe.prixfixe import PrixFixeNet


first = AutosomeFirstLayersBlock(in_channels=dataprocessor.data_channels(),
                                   out_channels=256, 
                                   seqsize=dataprocessor.data_seqsize())
core = AutosomeCoreBlock(in_channels=first.out_channels,
                         out_channels =64,
                         seqsize=first.infer_outseqsize())
final = AutosomeFinalLayersBlock(in_channels=core.out_channels, 
                                 seqsize=core.infer_outseqsize())
model = PrixFixeNet(
    first=first,
    core=core,
    final=final,
    generator=generator
)


Check if model works correctly

In [ ]:
model.check()

Model training is also simple 

In [ ]:
NUM_EPOCHS = 1
MODEL_LOG_DIR = "/home/penzard/autosome_model4"
CUDA_DEVICE_ID = 1

In [ ]:
import torch 
from prixfixe.autosome import AutosomeTrainer
trainer = AutosomeTrainer(
    model,    
    device=torch.device(f"cuda:{CUDA_DEVICE_ID}"), 
    model_dir=MODEL_LOG_DIR,
    dataprocessor=dataprocessor,
    num_epochs=NUM_EPOCHS)

In [ ]:
trainer.fit()